# Exploring Different Fitness / Selection Criterion

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')  # Replace '/path/to/EVO_BASE' with the actual path to the module
from EVO import LogisticRegression, EvolutionOptimizer
from fitness import FitnessOptimizer
import torch
import numpy as np
import time
import random
import matplotlib.pyplot as plt

In [2]:
def classification_data(n_points = 300, noise = 0.2, p_dims = 2):
    
    y = torch.arange(n_points) >= int(n_points/2)
    y = 1.0*y
    X = y[:, None] + torch.normal(0.0, noise, size = (n_points,p_dims))
    X = torch.cat((X, torch.ones((X.shape[0], 1))), 1)
    
    return X, y

In [3]:
def accuracy(model, X, y):
    y_pred = model.predict(X)
    return (y_pred == y).float().mean().item()

In [14]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("  GPU count:", torch.cuda.device_count())
    print("  Current device:", torch.cuda.current_device())
    print("  Device name:", torch.cuda.get_device_name(0))

Torch version: 2.5.1
CUDA available? False


In [12]:
X, y = classification_data(p_dims=200, noise=0.2, n_points=100)
LR = LogisticRegression()
opt = FitnessOptimizer(LR, "cuda" if torch.cuda.is_available() else "cpu")
opt.set_mutation_rate(0.5)
opt.set_population_size(100)

LR.set_optimizer(opt)

loss = []
start = time.time()
while accuracy(LR, X, y) < 1:
    opt.step(X, y, fitness_threshold=2)
    loss.append(LR.loss(X, y).item())
print("Time taken: ", time.time() - start)
plt.plot(loss, label="Evolution Optimizer Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Evolution Optimizer")
plt.legend()
plt.grid(True)
plt.show()

KeyboardInterrupt: 

In [ ]:
thresh_range = range(2, 10)
loss_dict = {}

for thresh in thresh_range:
    print(f"Running for thresh = {thresh}")
    
    # Generate new data for fairness (or keep it fixed if you prefer)
    X, y = classification_data(p_dims=200, noise=0.8, n_points=100)
    
    # Set up model and optimizer
    LR = LogisticRegression()
    opt = FitnessOptimizer(LR, "cuda")
    opt.set_mutation(0.5)
    opt.set_population_size(100)
    
    # Loss tracking
    loss = []
    start = time.time()
    
    # Optimization loop
    while accuracy(LR, X, y) < 1:
        opt.step(X, y, fitness_threshold=thresh)
        loss.append(LR.loss(X, y).item())
    
    print(f"Finished in {len(loss)} iterations, Time: {round(time.time() - start, 2)}s")
    loss_dict[thresh] = loss

# Plotting
plt.figure(figsize=(12, 8))
for k, loss in loss_dict.items():
    plt.plot(loss, label=f"Best 1/{k}")

plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.title("Evolution Optimizer Loss vs Survival Ratio")
plt.legend(title="Thresholds")
plt.grid(True)
plt.show()
